# Data Sources for Final Project

Looking at the following APIS:
- Fire detection areas
    - NASA Fire Information for Resource Management System: https://firms.modaps.eosdis.nasa.gov/api/
    - How to use the API: https://firms.modaps.eosdis.nasa.gov/content/academy/data_api/firms_api_use.html
- NDVI (Normalized Difference Vegetation Index) - used to assess “greenery” of an area over time
    - https://agromonitoring.com/api#ndvi
- General forest related data
    - Global Forest Watch - https://data-api.globalforestwatch.org/
- General weather data
    - Open Weather Map - https://openweathermap.org/api
- Burn impact and heat response through thermals
    - USGS Landsat data: https://m2m.cr.usgs.gov/

## Fire Detection Areas (FIRMS)

In [8]:
# Let's set your map key that was emailed to you. It should look something like 'abcdef1234567890abcdef1234567890'
MAP_KEY = '32f51992e2b4fc340806099364d917e3'

# check how many transactions we have
import pandas as pd
import requests
url = 'https://firms.modaps.eosdis.nasa.gov/mapserver/mapkey_status/?MAP_KEY=' + MAP_KEY
try:
  response = requests.get(url)
  data = response.json()
  df = pd.Series(data)
  display(df)
except:
  # possible error, wrong MAP_KEY value, check for extra quotes, missing letters
  print ("There is an issue with the query. \nTry in your browser: %s" % url)

transaction_limit             5000
current_transactions             0
transaction_interval    10 minutes
dtype: object

In [9]:
# let's create a simple function that tells us how many transactions we have used.
# We will use this in later examples

def get_transaction_count() :
  count = 0
  try:
    response = requests.get(url)
    data = response.json()
    df = pd.Series(data)
    count = df['current_transactions']
  except:
    print ("Error in our call.")
  return count

tcount = get_transaction_count()
print ('Our current transaction count is %i' % tcount)

Our current transaction count is 0


In [10]:
# let's query data_availability to find out what date range is available for various datasets
# we will explain these datasets a bit later

# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2025-06-01,2025-09-01
1,MODIS_SP,2000-11-01,2025-05-31
2,VIIRS_NOAA20_NRT,2025-04-01,2025-09-01
3,VIIRS_NOAA20_SP,2018-04-01,2025-03-31
4,VIIRS_NOAA21_NRT,2024-01-17,2025-09-01
5,VIIRS_SNPP_NRT,2025-04-01,2025-09-01
6,VIIRS_SNPP_SP,2012-01-20,2025-03-31
7,LANDSAT_NRT,2022-06-20,2025-08-31
8,GOES_NRT,2022-08-09,2025-09-01
9,BA_MODIS,2000-11-01,2025-05-01


In [11]:
# now let's see how many transactions we use by querying this end point

start_count = get_transaction_count()
pd.read_csv(da_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

# now remember, after 10 minutes this will reset

We used 5 transactions.


In [5]:
# in this example let's look at VIIRS NOAA-20, entire world and the most recent day
area_url = 'https://firms.modaps.eosdis.nasa.gov/api/area/csv/' + MAP_KEY + '/VIIRS_NOAA20_NRT/world/1'
start_count = get_transaction_count()
df_area = pd.read_csv(area_url)
end_count = get_transaction_count()
print ('We used %i transactions.' % (end_count-start_count))

df_area

We used 36 transactions.


,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,66.17818,158.37372,328.94,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,289.51,3.36,D
1,66.18291,158.39302,325.67,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,292.23,4.16,D
2,66.19170,158.39290,334.45,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,291.48,5.59,D
3,66.19747,158.34444,348.12,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,292.77,7.88,D
4,66.19904,158.36890,326.58,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,290.83,2.20,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85830,19.40579,-155.28601,327.86,0.8,0.80,2025-08-26,2235,N20,VIIRS,l,2.1URT,299.00,19.46,D
85831,19.40752,-155.27890,338.88,0.8,0.80,2025-08-26,2235,N20,VIIRS,n,2.1URT,302.35,21.54,D
85832,19.40928,-155.27168,339.10,0.8,0.80,2025-08-26,2235,N20,VIIRS,n,2.1URT,301.37,21.54,D
85833,19.41103,-155.26447,351.24,0.8,0.80,2025-08-26,2235,N20,VIIRS,n,2.1URT,301.39,28.36,D


In [12]:
# create dataframe with api data
import pandas as pd
df_area['acq_date'] = pd.to_datetime(df_area['acq_date'])

df_area.head()

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_ti5,frp,daynight
0,66.17818,158.37372,328.94,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,289.51,3.36,D
1,66.18291,158.39302,325.67,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,292.23,4.16,D
2,66.19170,158.39290,334.45,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,291.48,5.59,D
3,66.19747,158.34444,348.12,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,292.77,7.88,D
4,66.19904,158.36890,326.58,0.5,0.41,2025-08-26,48,N20,VIIRS,n,2.0NRT,290.83,2.20,D


## NDVI

In [ ]:
url_ndvi = "http://api.agromonitoring.com/agro/1.0/polygons?appid=b564ce3a3ea22838ef41c28da104aded"
